# 1: Import thư viện và thiết lập đường dẫn

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
from datetime import datetime

# Thêm thư mục src vào path để có thể import dbConnector
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from dbConnector import *

# 2: Khởi tạo kết nối Database

In [ ]:
# Đường dẫn đến file config (đảm bảo bạn đã sửa file này với thông tin thật)
config_path = os.path.join('..', 'config', 'db_config.yaml')

# Khởi tạo đối tượng dbJob
# Bạn có thể truyền tên DB cụ thể vào tham số thứ 2 nếu muốn override file config
db = dataInfo(config_path, databaseName=None)

print(f"Đã kết nối thành công! Server: {db.server}, Database: {db.database}")
print("Các bảng hiện có:", db.tableInDB)

# 3: Tạo dữ liệu giả lập (Mock Data)

In [ ]:
# Tạo một DataFrame mẫu về Doanh số bán hàng
data = {
    'id_date': [20230101, 20230102, 20230103], # Cột này thường dùng làm Primary Key trong code của bạn
    'product_name': ['Laptop', 'Mouse', 'Keyboard'],
    'quantity': [10, 50, 30],
    'price': ['15.5M', '200K', '500K'], # Dữ liệu dạng chuỗi cần làm sạch
    'created_at': [datetime.now(), datetime.now(), datetime.now()]
}
df = pd.DataFrame(data)
print("Dữ liệu mẫu:")
display(df)

# 4: Tạo bảng mới và Insert dữ liệu

In [ ]:
table_name = "Demo_Sales_Table"

# Nếu bảng đã tồn tại, xóa đi để demo lại từ đầu
if db.checkTableExist(table_name):
    print(f"Bảng {table_name} đã tồn tại, đang xóa...")
    db.dropTable(table_name)

# Tạo bảng mới và insert dữ liệu
# Hàm createNewTable sẽ tự động detect kiểu dữ liệu
db.createNewTable(df, table_name)

# Kiểm tra lại dữ liệu trong DB
df_result = db.getData(table_name)
display(df_result)

1. **Hàm** ```insertData```
Đây là hàm "All-in-one". Khi bạn gọi hàm này, quy trình sau diễn ra ngầm:

- **Kiểm tra bảng:** Nếu bảng chưa có -> Tạo bảng mới với kiểu dữ liệu tự động map từ Pandas sang SQL -> Set Primary Key.

- **Kiểm tra cột:** Nếu DataFrame có cột mới mà Database chưa có -> Tự động chạy lệnh ALTER TABLE ADD COLUMN.

- **Lọc trùng lặp (Quan trọng)**:
    - Hàm sẽ tải 2000 dòng mới nhất từ Database về bộ nhớ tạm (Cache).

    - So sánh DataFrame mới với Cache dựa trên primaryKey.

    - Nếu tìm thấy ID đã tồn tại, nó tiếp tục so sánh nội dung các cột khác.

    - Nếu nội dung y hệt -> Bỏ qua (Không Insert).

    - Nếu ID chưa có -> Insert.

- **Insert:** Sử dụng fast_executemany=True để đẩy dữ liệu xuống cực nhanh (nhanh hơn 50 lần so với insert thường).

> Tham số quan trọng:

    - dataTable: Pandas DataFrame chứa dữ liệu.

    - targetTableName: Tên bảng trong SQL Server.

    - primaryKey: Tên cột khóa chính (VD: 'id', 'date', 'id_date', 'transaction_code'). Bắt buộc phải có để tính năng lọc trùng hoạt động đúng.

2. **Hàm** ```getData```: Hàm cơ bản để lấy dữ liệu.

- ```columns```: Danh sách tên cột cần lấy (List[str]). Nếu để None sẽ lấy tất cả SELECT *.

**Lưu ý:** Nếu bảng dữ liệu rất lớn (> 100k dòng), hạn chế dùng getData mà hãy dùng getData_slice để tránh treo máy.

### 4.1 Demo insertData - Trường hợp 1: Tạo bảng mới

In [ ]:
# Tạo dữ liệu giả lập
data_batch_1 = {
    'transaction_id': [101, 102, 103],       # Sẽ dùng làm Primary Key
    'ticker': ['AAPL', 'MSFT', 'GOOGL'],
    'volume': [100, 200, 150],
    'price': [150.5, 280.0, 2500.0],
    'trade_date': [datetime(2023, 1, 1), datetime(2023, 1, 1), datetime(2023, 1, 1)]
}

df_batch_1 = pd.DataFrame(data_batch_1)
table_name = "DEMO_Transactions"

# Xóa bảng cũ nếu có để chạy demo cho sạch
if db.checkTableExist(table_name):
    db.dropTable(table_name)
    print(f"Đã xóa bảng cũ {table_name}")

print("Dữ liệu Batch 1:")
display(df_batch_1)

In [ ]:
print(f"--- Bắt đầu Insert vào {table_name} (Lần đầu) ---")

# Gọi hàm insertData
# primaryKey='transaction_id': Quan trọng để xác định duy nhất dòng dữ liệu
db.insertData(
    dataTable=df_batch_1, 
    targetTableName=table_name, 
    primaryKey='transaction_id'
)

# Kiểm tra kết quả
if db.checkTableExist(table_name):
    print("Insert thành công! Bảng đã được tạo.")
else:
    print("Insert thất bại.")

Demo ```getData``` - Lấy dữ liệu lên

In [ ]:
print("--- Lấy toàn bộ dữ liệu ---")
df_result = db.getData(table_name)
display(df_result)

print("--- Chỉ lấy cột ticker và price ---")
df_subset = db.getData(table_name, columns=['ticker', 'price'])
display(df_subset)

### 4.2 Demo insertData - Trường hợp 2: Insert thêm và Tự động loại bỏ trùng lặp

In [ ]:
# Tạo Batch 2: 
# ID 102 đã có trong DB (Trùng lặp) -> Cần bị bỏ qua
# ID 104 là mới -> Cần được thêm vào
data_batch_2 = {
    'transaction_id': [102, 104], 
    'ticker': ['MSFT', 'TSLA'],
    'volume': [200, 50],          # Giống hệt dữ liệu cũ của ID 102
    'price': [280.0, 700.0],
    'trade_date': [datetime(2023, 1, 1), datetime(2023, 1, 2)]
}
df_batch_2 = pd.DataFrame(data_batch_2)

print("Dữ liệu Batch 2 (Có chứa dòng trùng lặp ID 102):")
display(df_batch_2)

print(f"\n--- Bắt đầu Insert Batch 2 vào {table_name} ---")
db.insertData(
    dataTable=df_batch_2, 
    targetTableName=table_name, 
    primaryKey='transaction_id'
)

print("\n--- Kết quả sau khi Insert Batch 2 (Chỉ nên thêm ID 104) ---")
display(db.getData(table_name))

### 4.3 Demo insertData - Trường hợp 3: Schema Evolution (Tự thêm cột mới)

In [ ]:
# Tạo Batch 3: Có thêm cột mới 'sector'
data_batch_3 = {
    'transaction_id': [105],
    'ticker': ['NVDA'],
    'volume': [300],
    'price': [450.0],
    'trade_date': [datetime(2023, 1, 3)],
    'sector': ['Technology'] # <--- Cột mới chưa có trong DB
}
df_batch_3 = pd.DataFrame(data_batch_3)

print("Dữ liệu Batch 3 (Có cột mới 'sector'):")
display(df_batch_3)

print(f"\n--- Bắt đầu Insert Batch 3 ---")
# Module sẽ tự phát hiện cột mới và Alter Table
db.insertData(
    dataTable=df_batch_3, 
    targetTableName=table_name, 
    primaryKey='transaction_id'
)

print("\n--- Kết quả: Cột 'sector' đã được thêm vào và các dòng cũ giá trị là NULL ---")
display(db.getData(table_name))

### 4.4 Demo getData_slice - Lấy dữ liệu nâng cao

In [ ]:
print("--- Lấy Top 2 dòng mới nhất (theo transaction_id) ---")
# arg_choice=2 (số lượng), sort_by='transaction_id', is_exact=False (nghĩa là lấy Top)
df_top = db.getData_slice(2, table_name, sort_by='transaction_id', is_exact=False)
display(df_top)

print("--- Lấy chính xác dòng có transaction_id > 103 ---")
# arg_choice="> 103" (điều kiện), is_exact=True (nghĩa là dùng WHERE)
df_filter = db.getData_slice("> 103", table_name, sort_by='transaction_id', is_exact=True)
display(df_filter)

# 5: Demo Insert thông minh (Tránh trùng lặp)

In [ ]:
# Tạo dữ liệu mới, trong đó có 1 dòng cũ (Laptop) và 1 dòng mới (Monitor)
new_data = {
    'id_date': [20230101, 20230104], 
    'product_name': ['Laptop', 'Monitor'],
    'quantity': [10, 5],
    'price': ['15.5M', '3.5M'],
    'created_at': [datetime.now(), datetime.now()]
}
df_new = pd.DataFrame(new_data)

# Gọi hàm insertData
# Nó sẽ tự động bỏ qua dòng Laptop (vì trùng id_date) và chỉ thêm Monitor
db.insertData(df_new, table_name, primaryKey='id_date')

print("Dữ liệu sau khi insert thêm:")
display(db.getData(table_name))

# 6: Demo Sync dữ liệu (Update + Insert)

In [ ]:
# Giả sử dòng Mouse (20230102) thay đổi số lượng từ 50 -> 100
# Và có thêm dòng Headphone (20230105)
update_data = {
    'id_date': [20230102, 20230105], 
    'product_name': ['Mouse', 'Headphone'],
    'quantity': [100, 20], # Mouse thay đổi
    'price': ['200K', '1.2M'],
    'created_at': [datetime.now(), datetime.now()]
}
df_update = pd.DataFrame(update_data)

# Sử dụng check_and_update_table
# key_columns: Cột dùng để định danh dòng (thường là Primary Key)
# numeric_cols: Cột cần làm sạch số liệu (ví dụ: '200K' -> 200000)
check_and_update_table(
    df_new=df_update, 
    table_name=table_name, 
    engine=db.engine, 
    key_columns=['id_date'], 
    numeric_cols=['price']
)

print("Dữ liệu sau khi Sync:")
display(db.getData(table_name))